# Transfer Learning Example on Radar Dataset (Ouachita Mountains)

## Importing the required modules

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)
import scipy.io as sio
import pandas as pd
import os
import re
import math
import itertools
import time
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold
from itertools import product
from sklearn.utils.class_weight import compute_class_weight
torch.cuda.empty_cache()

# Reading the labels and creating train and test data

In [2]:
def index(string):
    s = re.findall("[0-9]", string)
    return int(''.join(s))

scenario_idx = 29 # Bonneville Salt Flats
names = os.listdir(f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/')
names = sorted(names, key=index)
print(len(names))
y = pd.read_csv(f'num{scenario_idx}_Ground_Truth_20_rng30_1000_100k_m.csv')
col_names = y.columns[4:7]
y = y[col_names].to_numpy()

scenario_idx_test = 62 # Ouachita Mountains
names_test = os.listdir(f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/')
names_test = sorted(names_test, key=index)
y_test = pd.read_csv(f'num{scenario_idx_test}_Ground_Truth_20_rng30_1000_100k_m.csv')
col_names_test = y_test.columns[4:7]
y_test = y_test[col_names_test].to_numpy()

# Define fine-tuning name and labels
few_shot_count = 64
names_FS = names_test[:few_shot_count]
y_FS = y_test[:few_shot_count]

# Create training and testing datasets
y_train = y[:int(0.9 * len(names))]
y_test = y_test[int(0.9 * len(names_test)) - 1:]
training_names = names[:int(0.9 * len(names))]
test_names = names_test[int(0.9 * len(names_test)) - 1:]

print('Training labels:')
print(y_train)

# Tensor Corners
##################################################################################
# num29: [10851, 215, -5.4], num60: [11073, 215, -5.3], num62: [11471, 215, -5.6]
# num76: [11388, 215, -6.15], num35: [11381, 215, -0.95]
##################################################################################

# Training dataset global constants
coord_tr = [10851, 215, -5.4]  # Tensor corner
rng_res_tr = 59.9585 / 2        # Range resolution
az_step_tr = 0.4                # Azimuth step size
el_step_tr = 0.01               # Elevation step size

# Test dataset global constants
coord_ts = [11471, 215, -5.6]  # Tensor corner
rng_res_ts = 59.9585 / 2        # Range resolution
az_step_ts = 0.4                # Azimuth step size
el_step_ts = 0.01               # Elevation step size

def Drawing_Batch(names, label, bs, ind, directory, normalize=True):
    x = []
    labels = []
    
    for j in range(ind * bs, (ind + 1) * bs):
        try:
            temp = sio.loadmat(directory + names[j])['P']
        except:
            break
        if normalize:
            Anorm = temp - np.min(temp.flatten())
            temp = np.divide(Anorm, np.max(Anorm.flatten()))
        x.append(temp)
        labels.append(label[j, :])
        
    x = torch.FloatTensor(np.array(x))
    labels = torch.FloatTensor(np.array(labels))
    return x, labels

100000
Training labels:
[[ 8.3355 13.325  19.525 ]
 [18.156  17.388  32.775 ]
 [ 8.777  24.73   18.582 ]
 ...
 [15.785  15.992  29.641 ]
 [ 3.1736 10.716  12.27  ]
 [ 4.5552 21.035  14.234 ]]


In [3]:
print(len(y_train))
print(len(y_test))

90000
10001


## Define a regression CNN and instantiating it

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(21, 32, 3, 1)
        self.conv2 = nn.Conv1d(32, 64, 3, 1)
        self.batchnorm1 = nn.BatchNorm1d(32)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.fc1 = nn.Linear(64 * 5, 20)  # Adjust input size based on the output of conv layers and max pooling
        self.fc2_reg = nn.Linear(20, 2)  # Adjusted output size

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.batchnorm1(x))
        x = F.max_pool1d(x, 2)
        
        x = self.conv2(x)
        x = F.relu(self.batchnorm2(x))
        x = F.max_pool1d(x, 2)
        
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        output_reg = self.fc2_reg(x)  # (bs, 2)
        
        return output_reg
    
from torchsummary import summary
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = Net()
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
print(summary(model,(21,26)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 24]           2,048
       BatchNorm1d-2               [-1, 32, 24]              64
            Conv1d-3               [-1, 64, 10]           6,208
       BatchNorm1d-4               [-1, 64, 10]             128
            Linear-5                   [-1, 20]           6,420
            Linear-6                    [-1, 2]              42
               Net-7                    [-1, 2]               0
Total params: 14,910
Trainable params: 14,910
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.06
Estimated Total Size (MB): 0.08
----------------------------------------------------------------
None


## Global Definitions

In [5]:
bs = 128  # batch_size
num_epoch = 50  # number of epochs
PATH = './ckpt_model.pth'   # forsaving the model
criterion = nn.MSELoss()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Define a Loss function and optimizer; Using GPU or CPU
model = Net()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
    
def Spher2Cart_1D(spherical):
    cartesian = np.zeros(3)
    hypotenuse = np.cos(np.radians(spherical[2]))*spherical[0]
    cartesian[0] = np.cos(np.radians(spherical[1]))*hypotenuse
    cartesian[1] = -np.sin(np.radians(spherical[1]))*hypotenuse
    cartesian[2] = np.sin(np.radians(spherical[2]))*spherical[0]
    return cartesian

## Train and evaluate the network

In [6]:
def main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True):
    best_error = 1e+20  # a dummy and very large number for saving the best discovered model
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-' * 10)
        running_loss_train = 0
        running_loss_test = 0

        model.train()
        for i in range(0, len(training_names) // bs):
            x_train, labels = Drawing_Batch(training_names, y_train, bs, i, f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
            x_train = x_train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x_train)
            loss = criterion(out, labels[:, 0:2])
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            
        out = torch.cat((out, torch.unsqueeze(labels[:,2], dim=1)), dim=1)

        # Training error display
        true_train = Spher2Cart_1D(np.multiply(labels.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        pred_train = Spher2Cart_1D(np.multiply(out.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        print(true_train)
        print(pred_train)

        model.eval()
        with torch.no_grad():
            for i in range(0, len(test_names) // bs):
                x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
                x_test = x_test.to(device)
                labels_test = labels_test.to(device)
                out_test = model(x_test)
                loss_test = criterion(out_test, labels_test[:, 0:2])
                running_loss_test += loss_test.item()
                
        out_test = torch.cat((out_test, torch.unsqueeze(labels_test[:,2], dim=1)), dim=1)

        # Test error display
        true_test = Spher2Cart_1D(np.multiply(labels_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        pred_test = Spher2Cart_1D(np.multiply(out_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        print(true_test)
        print(pred_test)

        epoch_loss_train = running_loss_train * x_train.size(0) / len(training_names)
        epoch_loss_test = running_loss_test * x_test.size(0) / len(test_names)

        print('Train Loss: {:.6f} ---- Test Loss: {:.6f}'.format(epoch_loss_train, epoch_loss_test))
        if epoch % 5 == 0:
            if epoch_loss_test < best_error:
                torch.save(model.state_dict(), PATH)
                best_error = epoch_loss_test

start = time.time()
main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True)
end = time.time()
print(end - start)

Epoch 0/50
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8205.89466346  7729.9688152  -1006.03124537]
[-8209.36050705  7974.67245585 -1062.87089418]
[-8673.48676275  7538.50656094 -1067.20095188]
Train Loss: 8.731725 ---- Test Loss: 67.093027
Epoch 1/50
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8195.82266503  7745.72584678 -1006.34250361]
[-8209.36050705  7974.67245585 -1062.87089418]
[-8628.45446684  7610.61948904 -1068.46614085]
Train Loss: 0.185318 ---- Test Loss: 58.223908
Epoch 2/50
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8197.49127336  7759.01678775 -1007.26569128]
[-8209.36050705  7974.67245585 -1062.87089418]
[-8612.02345273  7641.13144918 -1069.2007458 ]
Train Loss: 0.130282 ---- Test Loss: 56.654466
Epoch 3/50
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8201.53093511  7752.35814815 -1007.1192642 ]
[-8209.36050705  7974.67245585 -1062.87089418]
[-8594.87771448  7660.26664837 -1069.19175311]
Train Loss: 0.11351

[-8187.45955885  7784.83170223 -1008.20172776]
[-8197.81612099  7768.24910298 -1007.85326491]
[-8209.36050705  7974.67245585 -1062.87089418]
[-8506.23277849  7796.34327211 -1071.55801079]
Train Loss: 0.061774 ---- Test Loss: 31.059137
Epoch 33/50
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8197.42319634  7768.67607146 -1007.85402222]
[-8209.36050705  7974.67245585 -1062.87089418]
[-8507.64462748  7795.97984028 -1071.63186969]
Train Loss: 0.060901 ---- Test Loss: 30.747501
Epoch 34/50
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8197.76255035  7768.22879488 -1007.84854826]
[-8209.36050705  7974.67245585 -1062.87089418]
[-8507.87631973  7796.82243039 -1071.70059908]
Train Loss: 0.060055 ---- Test Loss: 30.409379
Epoch 35/50
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8197.3003067   7768.87909916 -1007.8585254 ]
[-8209.36050705  7974.67245585 -1062.87089418]
[-8509.26904553  7796.80552408 -1071.7948961 ]
Train Loss: 0.059098 ---- Test Loss: 3

In [7]:
def Spher2Cart_2D(spherical):
    cartesian = np.zeros((len(spherical), 3))
    hypotenuse = np.multiply(np.cos(np.radians(spherical[:, 2])), spherical[:, 0])
    cartesian[:, 0] = np.multiply(np.cos(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 1] = np.multiply(-np.sin(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 2] = np.multiply(np.sin(np.radians(spherical[:, 2])), spherical[:, 0])
    return cartesian

# Testing: (range, az, el)
model.eval()
out_test_reg = np.zeros((len(y_test), 3))
labels_test_reg = np.zeros((len(y_test), 3))

for i in range(0, len(y_test) // bs):
    x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', True)
    x_test = x_test.to(device)
    labels_test = labels_test.cpu().data.numpy()
    labels_test_reg[bs * i: bs * i + bs] = (labels_test[:, 0:3])

    cur_test_reg = model(x_test)
    out_test_reg[bs * i: bs * i + bs, 0:2] = cur_test_reg.cpu().data.numpy()
    out_test_reg[bs * i: bs * i + bs, 2] = labels_test_reg[bs * i: bs * i + bs, 2]

# Calculate azimuth estimation error
azim_tot = 0
for i in range(len(out_test_reg)):
    azim_tot += np.linalg.norm(out_test_reg[i, 1] - labels_test_reg[i, 1])

azim_tot = azim_tot / len(out_test_reg)
print(f'Azimuth Estimation Error (deg) = {azim_tot}')

# Convert spherical coordinates to Cartesian coordinates
new_data = Spher2Cart_2D(np.multiply(out_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
new_labels_data = Spher2Cart_2D(np.multiply(labels_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)

# Calculate localization error
sum_tot = 0
for i in range(0, len(new_data) - (len(new_data) % bs), 1):
    sum_tot += np.linalg.norm(new_data[i, :] - new_labels_data[i, :])

sum_tot = sum_tot / (len(new_data) - (len(new_data) % bs))
print(f'Localization Error (m) = {sum_tot}')

Azimuth Estimation Error (deg) = 6.152806818050189
Localization Error (m) = 511.4853873152036


## Fine-Tuning Regression CNN

In [8]:
model.module.conv1.weight.requires_grad = False
model.module.conv1.bias.requires_grad = False
model.module.batchnorm1.weight.requires_grad = False
model.module.batchnorm1.bias.requires_grad = False

model.module.conv2.weight.requires_grad = False
model.module.conv2.bias.requires_grad = False
model.module.batchnorm2.weight.requires_grad = False
model.module.batchnorm2.bias.requires_grad = False

In [9]:
bs = 64  # batch_size
num_epoch = 100
optimizer = optim.Adam(model.parameters(), lr = 0.001)

def main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True):
    best_error = 1e+20  # a dummy and very large number for saving the best discovered model
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-' * 10)
        running_loss_train = 0
        running_loss_test = 0

        model.train()
        for i in range(0, len(training_names) // bs):
            x_train, labels = Drawing_Batch(training_names, y_train, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
            x_train = x_train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x_train)
            loss = criterion(out, labels[:, 0:2])
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            
        out = torch.cat((out, torch.unsqueeze(labels[:,2], dim=1)), dim=1)

        # Training error display
        true_train = Spher2Cart_1D(np.multiply(labels.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        pred_train = Spher2Cart_1D(np.multiply(out.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        print(true_train)
        print(pred_train)

        model.eval()
        with torch.no_grad():
            for i in range(0, len(test_names) // bs):
                x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
                x_test = x_test.to(device)
                labels_test = labels_test.to(device)
                out_test = model(x_test)
                loss_test = criterion(out_test, labels_test[:, 0:2])
                running_loss_test += loss_test.item()
                
        out_test = torch.cat((out_test, torch.unsqueeze(labels_test[:,2], dim=1)), dim=1)

        # Test error display
        true_test = Spher2Cart_1D(np.multiply(labels_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        pred_test = Spher2Cart_1D(np.multiply(out_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        print(true_test)
        print(pred_test)

        epoch_loss_train = running_loss_train * x_train.size(0) / len(training_names)
        epoch_loss_test = running_loss_test * x_test.size(0) / len(test_names)

        print('Train Loss: {:.6f} ---- Test Loss: {:.6f}'.format(epoch_loss_train, epoch_loss_test))
        if epoch % 5 == 0:
            if epoch_loss_test < best_error:
                torch.save(model.state_dict(), PATH)
                best_error = epoch_loss_test

main(names_FS, test_names, bs, num_epoch, y_FS, y_test, normalize=True)

Epoch 0/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-8342.33102062  7299.63588013 -1000.00141727]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8486.41324285  7815.04882915 -1071.67841562]
Train Loss: 21.412037 ---- Test Loss: 26.475146
Epoch 1/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-8326.1469946   7314.88980451  -999.81082865]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8461.55583635  7835.91001774 -1071.29674204]
Train Loss: 19.837910 ---- Test Loss: 24.557225
Epoch 2/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-8309.4573078   7330.33690282  -999.60154979]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8438.30445479  7855.92490089 -1070.97903045]
Train Loss: 18.338196 ---- Test Loss: 22.710969
Epoch 3/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-8292.26205663  7345.97573213  -999.3737682 ]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8419.37979373  7870.46156041 -1070.61471125]
Train Loss: 

[-8202.70901443  7975.78315676 -1062.79789273]
[-8224.83938222  8003.91961779 -1066.09393098]
Train Loss: 4.788298 ---- Test Loss: 5.338945
Epoch 33/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-7948.73382216  7698.39457862  -998.24398961]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8230.48969888  7996.99543543 -1066.02182763]
Train Loss: 4.663462 ---- Test Loss: 5.232251
Epoch 34/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-7945.79992442  7701.28198472  -998.2351531 ]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8236.19857025  7989.88853039 -1065.9424513 ]
Train Loss: 4.546553 ---- Test Loss: 5.130881
Epoch 35/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-7943.08709584  7703.74204993  -998.21393007]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8241.89778426  7982.71692857 -1065.85891619]
Train Loss: 4.438734 ---- Test Loss: 5.036019
Epoch 36/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-7940.55830035

[-8202.70901443  7975.78315676 -1062.79789273]
[-8291.00408142  7924.37984563 -1065.38882381]
Train Loss: 3.614155 ---- Test Loss: 4.535872
Epoch 65/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-7876.72071154  7762.59327824  -997.64434252]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8289.98092872  7925.1206718  -1065.36767096]
Train Loss: 3.608359 ---- Test Loss: 4.538505
Epoch 66/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-7875.32555798  7763.48323615  -997.61106433]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8289.03938285  7925.79176975 -1065.34753326]
Train Loss: 3.602851 ---- Test Loss: 4.540433
Epoch 67/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-7874.14277744  7764.21766786  -997.58159817]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8288.19212091  7926.38985095 -1065.3290477 ]
Train Loss: 3.597513 ---- Test Loss: 4.541602
Epoch 68/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-7873.17693999

[-8202.70901443  7975.78315676 -1062.79789273]
[-8276.41714944  7932.71794894 -1064.94548397]
Train Loss: 3.461897 ---- Test Loss: 4.426812
Epoch 97/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-7881.5784099   7757.00123097  -997.60258745]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8275.71507468  7933.31139972 -1064.93654993]
Train Loss: 3.458824 ---- Test Loss: 4.426358
Epoch 98/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-7881.65910211  7756.96288106  -997.60534885]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8275.02087348  7933.89990093 -1064.92783241]
Train Loss: 3.455849 ---- Test Loss: 4.426041
Epoch 99/100
----------
[-7973.8157493   7783.385959   -1005.21113055]
[-7881.71954117  7756.93910122  -997.60773009]
[-8202.70901443  7975.78315676 -1062.79789273]
[-8274.33922898  7934.47398584 -1064.91903654]
Train Loss: 3.452971 ---- Test Loss: 4.425775


In [10]:
def Spher2Cart_2D(spherical):
    cartesian = np.zeros((len(spherical), 3))
    hypotenuse = np.multiply(np.cos(np.radians(spherical[:, 2])), spherical[:, 0])
    cartesian[:, 0] = np.multiply(np.cos(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 1] = np.multiply(-np.sin(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 2] = np.multiply(np.sin(np.radians(spherical[:, 2])), spherical[:, 0])
    return cartesian

# Testing: (range, az, el)
model.eval()
out_test_reg = np.zeros((len(y_test), 3))
labels_test_reg = np.zeros((len(y_test), 3))

for i in range(0, len(y_test) // bs):
    x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', True)
    x_test = x_test.to(device)
    labels_test = labels_test.cpu().data.numpy()
    labels_test_reg[bs * i: bs * i + bs] = (labels_test[:, 0:3])

    cur_test_reg = model(x_test)
    out_test_reg[bs * i: bs * i + bs, 0:2] = cur_test_reg.cpu().data.numpy()
    out_test_reg[bs * i: bs * i + bs, 2] = labels_test_reg[bs * i: bs * i + bs, 2]

# Calculate azimuth estimation error
azim_tot = 0
for i in range(len(out_test_reg)):
    azim_tot += np.linalg.norm(out_test_reg[i, 1] - labels_test_reg[i, 1])

azim_tot = azim_tot / len(out_test_reg)
print(f'Azimuth Estimation Error (deg) = {azim_tot}')

# Convert spherical coordinates to Cartesian coordinates
new_data = Spher2Cart_2D(np.multiply(out_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
new_labels_data = Spher2Cart_2D(np.multiply(labels_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)

# Calculate localization error
sum_tot = 0
for i in range(0, len(new_data) - (len(new_data) % bs), 1):
    sum_tot += np.linalg.norm(new_data[i, :] - new_labels_data[i, :])

sum_tot = sum_tot / (len(new_data) - (len(new_data) % bs))
print(f'Localization Error (m) = {sum_tot}')

Azimuth Estimation Error (deg) = 0.7389588330748224
Localization Error (m) = 97.71598280344698
